In [8]:
# 初始化環境
from rag_to_vector_db_pinecone.rag_pipeline.chunks_embeding_2_vector_db import (
    jina_embedding,
)

import json
from pinecone import Pinecone
from langchain_openai import ChatOpenAI

# 載入 env 設定
with open('env.json', 'r', encoding='utf-8') as file:
    config = json.load(file)

# 初始化 Pinecone gRPC 客戶端
pc = Pinecone(api_key=config['pinecone_api_key'])
index = pc.Index("rag-vector-db-test")

# 初始化 OpenAI 模型
llm = ChatOpenAI(
    model="gpt-3.5-turbo",  # 可以改為 gpt-3.5-turbo 測試
    openai_api_key=config["chat_gpt_key"],
    temperature=0.6
)

In [13]:
# 使用者輸入查詢 pinecone
def query_pincone(query, config):
    query_embedding = jina_embedding(query, config)
    results = index.query(
        vector=query_embedding,
        top_k=3,
        include_metadata=True
    )

    matches = results['matches']
    # 用 id 匹配資料庫文字
    with open('data/chunks.json', 'r', encoding='utf-8') as file:
        chunks = json.load(file)

    match_chunks_data = []
    for match in matches:
        for chunk in chunks:
            if chunk['id'] == match['id']:
                match_chunks_data.append({
                    "id" : match['id'],
                    "score" : match['score'],
                    "text" : chunk['text']
                })
                break

    return match_chunks_data


def create_augmented_prompt(query, match_chunks_data):
    # 將檢索到的上下文整合成一個字符串
    contexts = ""
    for i, context in enumerate(match_chunks_data):
        contexts += f"參考資料 {i+1} [相關度: {context['score']:.2f}]:\n{context['text']}\n\n"
    
    # 構建結構化提示
    prompt = f"""
    以下是使用者的查詢問題，請根據提供的參考資料回答。如果參考資料中沒有相關資訊，請明確說明。

    使用者查詢: {query}
    請根據以下參考資料提供準確、簡潔且有條理的回答。引用資料時請註明來源編號。

    {contexts}
    """
    
    return prompt


def query_llm(prompt):
    try:
        response = llm.invoke(prompt)
        return response.content
    except:
        return f"LLM 錯誤"

In [15]:
def main():
    # 1. 決定使用者輸入
    query = '黴菌相關風險與控制'

    # 2. 尋找知識庫
    match_chunks_data = query_pincone(query, config)

    # 3. 將 知識庫資料結合至 prompt
    prompt = create_augmented_prompt(query, match_chunks_data)
    
    # 4. prompt 引入 LLM
    rag_result = query_llm(prompt)
    result = query_llm(query)       # 沒有經過 RAG 檢索
    
    # 5. 對比rag 與沒有 rag 的狀況
    print('================ RAG result ==========')
    print(rag_result)
    
    print('================ 普通 result ==========')
    print(result)

main()

================ RAG result ==========
根據參考資料1和參考資料2提供的資訊，黴菌相關風險與控制可以從以下幾個方面進行探討：

1. 黴菌毒素的主要類型包括黃曲霉毒素、赭曲霉毒素、鐮刀菌毒素、單端孢霉烯族毒素和玉米赤霉烯酮，這些毒素由不同的黴菌產生。

2. 檢測方法包括色譜分析（如HPLC、GC-MS）、免疫分析（如ELISA）和生物感測器技術。

3. 防控策略可以從預防和去除兩個方面進行：
   - 預防措施包括控制儲存條件、使用防黴包裝、實施農業良好實踐。
   - 去除方法包括物理方法（如加熱處理、輻照、吸附）、化學方法（如氧化劑、還原劑、鹼處理）和生物方法（如微生物或酶降解）。

總結來說，對於黴菌相關風險的控制，除了了解不同毒素的來源和檢測方法外，採取適當的防控策略也是至關重要的。進一步的研究與發展趨勢將有助於更好地利用黴菌資源並解決相關問題。
================ 普通 result ==========
黴菌是一種微生物，常見於環境中的空氣、土壤、水和食物中。雖然有些黴菌對人體無害，但有些黴菌則可能對人體健康構成風險。黴菌感染可能導致呼吸道問題、皮膚炎症、過敏反應和其他健康問題。

以下是一些常見的黴菌相關風險和控制方法：

1. 室內空氣品質：黴菌常常在潮濕和通風不良的環境中生長，因此要保持室內空氣乾燥和通風良好是很重要的。定期清潔和除濕也能幫助減少黴菌的生長。

2. 食品安全：黴菌可能在食物中生長，尤其是在潮濕和溫暖的環境中。為了避免食品中的黴菌污染，應保持食品儲存和處理的衛生乾淨，並避免食用已經發霉的食物。

3. 職業風險：一些工作環境中，如農業、建築和清潔行業，可能暴露於高濃度的黴菌，增加了工作者患上黴菌相關疾病的風險。在這些環境中，應該提供適當的防護措施，如呼吸器和防護服。

4. 應急處理：如果發現室內有大面積的黴菌生長，應及時清除並修復引起潮濕的問題，以防止進一步的生長和擴散。對於嚴重的黴菌污染，可能需要專業的清潔公司來處理。

總的來說，要預防黴菌相關風險，需要保持環境清潔、乾燥和通風良好，避免暴露於高濃度的黴菌，並在需要時採取適當的防護措施和應急處理。
